In [3]:
%matplotlib notebook
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from utils.mri_viewer import MRIViewer
from utils.data_utils import im_path_to_arr
from utils.config import Config
from utils.dice_score import dice_score
from utils.data_utils import resize_data_to_original_size
from models.fcn_concat import FCN_Concat
from models.fcn_concat_v2 import FCN_Concat_v2

In [2]:
config_path = 'config_files/fcn_train_concat_2017_v691.cfg'
config = Config(config_path)
# model = FCN_Concat(config)
model = FCN_Concat_v2(config)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 88, in _path_is_mode_type
  File "<frozen importlib._bootstrap_external>", line 82, in _path_stat
FileNotFoundError: [Errno 2] No such file or directory: '/home/marcthib/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/contrib/cudnn_rnn/__init__.cpython-36m-x86_64-linux-gnu.so'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marcthib/miniconda3/envs/myenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-3e6e8c8d5d8a>", line 4, in <module>
    model = FCN_Concat_v2(config)
  File "/home/marcthib/research/romain/models/fcn.py", line 19, in __init__
    self.add_dataset()
  File "/home/marcthib/research/romain/models/fcn_concat.py", line 32, in add_dataset
    train_dataset = get_dataset_v3(self.config.tra

TypeError: must be str, not list

In [4]:
# for BraTS
# between 1 and 47
patient_id = 11

In [5]:
# for BraTS
ex_path = '/local-scratch/marcthib_scratch/brats2017/val/'
patient = os.listdir(ex_path)[patient_id - 1]
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, patient + '_seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, patient + '_flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, patient + '_t1ce.nii')
t1post = im_path_to_arr(t1post_path)

In [5]:
# for Rembrant
# between 1 and 13
patient_id = 5

In [6]:
# for Rembrandt
ex_path = '/local-scratch/romains_scratch/rembrandt/val/'
patient = os.listdir(ex_path)[patient_id - 1]
patient = 'HF1587='
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, 'seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, 'flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, 't1c.nii')
t1post = im_path_to_arr(t1post_path)

### select a patient id between 1 and 193

In [7]:
patient_id = 10

In [4]:
# for RTOG
ex_path = '/local-scratch/romain_scratch/rtog/'
patient = os.listdir(ex_path)[patient_id - 1]
patient_path = os.path.join(ex_path, patient)
flair_path = os.path.join(patient_path, 'flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, 't1c.nii')
t1post = im_path_to_arr(t1post_path)
print('shape of this patient is:', t1post.shape)

import gzip

def j(path, fname):
    return os.path.join(path, fname)

patient_path = '/home/romains/rtog_ex2/post'

with gzip.open(j(patient_path, 'flair.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 'flair.nii'), 'wb') as f:
                f.write(fgz.read())
                
with gzip.open(j(patient_path, 't1c.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 't1c.nii'), 'wb') as f:
                f.write(fgz.read())

t1post = im_path_to_arr(j(patient_path, 't1c.nii'))
flair = im_path_to_arr(j(patient_path, 'flair.nii'))

FileNotFoundError: [Errno 2] No such file or directory: '/local-scratch/romain_scratch/rtog/'

In [9]:
# for RTOG
ex_path = '/local-scratch/marcthib_scratch/rtog/'
patient = os.listdir(ex_path)[patient_id - 1]
patient_path = os.path.join(ex_path, patient)
flair_path = os.path.join(patient_path, '642flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, '642t1c.nii')
t1post = im_path_to_arr(t1post_path)
print('shape of this patient is:', t1post.shape)

import gzip

def j(path, fname):
    return os.path.join(path, fname)

patient_path = '/home/romains/rtog_ex2/post'

with gzip.open(j(patient_path, 'flair.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 'flair.nii'), 'wb') as f:
                f.write(fgz.read())
                
with gzip.open(j(patient_path, 't1c.nii.gz'), 'rb') as fgz:
            with open(j(patient_path, 't1c.nii'), 'wb') as f:
                f.write(fgz.read())

t1post = im_path_to_arr(j(patient_path, 't1c.nii'))
flair = im_path_to_arr(j(patient_path, 'flair.nii'))

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:89:
sitk::ERROR: The file "/local-scratch/marcthib_scratch/rtog/306t1c.nii/642flair.nii" does not exist.

In [ ]:
patient_path = patient_path.encode('utf-8')
saver = tf.train.Saver()
conf = tf.ConfigProto()
conf.gpu_options.allow_growth = True
with tf.Session() as sess:
    saver.restore(sess, config.ckpt_path)
    pred = model.run_pred_single_example_v3(sess, patient_path)
    pred[pred == 3] = 4
patient_path = patient_path.decode('utf-8')

In [ ]:
# for Rembrandt and RTOG
# resize to original size
original_size = im_path_to_arr(os.path.join(patient_path, 'flair.nii')).shape
pred = resize_data_to_original_size(pred, original_size)

In [7]:
# for Brats and Rembrandt
def get_dice_scores(p, y):
    dice_whole = dice_score(y, pred)
    fpred_core = (p == 1) + (p == 4)
    fy_core = (y == 1) + (y == 4)
    dice_core = dice_score(fy_core, fpred_core)
    fpred_enhancing = p == 4
    fy_enhancing = y == 4
    dice_enhancing = dice_score(fy_enhancing, fpred_enhancing)
    print('whole: %f ... core: %f ... enhancing: %f ...'%(dice_whole, dice_core, dice_enhancing))
get_dice_scores(pred, label)

whole: 0.903463 ... core: 0.899811 ... enhancing: 0.753856 ...


In [ ]:
# for Brats and Rembrandt
flair_pred = np.array(flair)
flair_pred[pred == 1] = 1000
flair_pred[pred == 2] = 2000
flair_pred[pred == 4] = 4000
flair_label = np.array(flair)
flair_label[label == 1] = 1000
flair_label[label == 2] = 2000
flair_label[label == 4] = 4000
MRIViewer(flair_pred, flair_label, flair, t1post, cmap=None, title1='prediction', title2='ground truth', title3='flair', title4='t1post')

In [9]:
print(np.count_nonzero(label == 1), np.count_nonzero(pred == 1))
print(np.count_nonzero(label == 2), np.count_nonzero(pred == 2))
print(np.count_nonzero(label == 4), np.count_nonzero(pred == 4))

10430 3691
67269 66555
23187 28677


### legend:
- yellow = 2000 = EDEMA
- green = 4000 = ENHANCING
- blue = 1000 = NECROSYS

# Vizualize patients 

In [ ]:
# for Rembrandt
patient_id = 5
ex_path = '/local-scratch/romains_scratch/rembrandt/val/'
patient = os.listdir(ex_path)[patient_id - 1]
patient = 'HF1702='
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, 'seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, 'flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, 't1c.nii')
t1post = im_path_to_arr(t1post_path)
flair_label = np.array(flair)
flair_label[label == 1] = 1000
flair_label[label == 2] = 2000
flair_label[label == 4] = 4000
MRIViewer(flair_label, flair_label, flair, t1post, cmap=None, title1='ground truth', title2='ground truth', title3='flair', title4='t1post')

In [ ]:
print(np.count_nonzero(label == 1))
print(np.count_nonzero(label == 2))
print(np.count_nonzero(label == 4))

In [ ]:
# for Brats
patient_id = 15
ex_path = '/local-scratch/romains_scratch/brats2017/val/'
patient = os.listdir(ex_path)[patient_id - 1]
patient_path = os.path.join(ex_path, patient)
label_path = os.path.join(patient_path, patient + '_seg.nii')
label = im_path_to_arr(label_path)
flair_path = os.path.join(patient_path, patient + '_flair.nii')
flair = im_path_to_arr(flair_path)
t1post_path = os.path.join(patient_path, patient + '_t1ce.nii')
t1post = im_path_to_arr(t1post_path)
flair_label = np.array(flair)
flair_label[label == 1] = 1000
flair_label[label == 2] = 2000
flair_label[label == 4] = 4000
MRIViewer(flair_label, flair_label, flair, t1post, cmap=None, title1='ground truth', title2='ground truth', title3='flair', title4='t1post')

In [ ]:
print(np.count_nonzero(label == 1))
print(np.count_nonzero(label == 2))
print(np.count_nonzero(label == 4))

In [ ]:
# for RTOG
flair_pred = np.array(flair)
flair_pred[pred == 1] = 1000
flair_pred[pred == 2] = 2000
flair_pred[pred == 4] = 4000
MRIViewer(flair_pred, flair_pred, flair, t1post, cmap=None, title1='prediction', title2='prediction', title3='flair', title4='t1post')